In [10]:
#VGG的一个关键是使用很多有着相对小的kernel（3×33×3）的卷积层然后接上一个池化层，之后再将这个模块重复多次。
from mxnet.gluon import nn

def vgg_block(num_convs, channels):
    out = nn.Sequential()
    for _ in range(num_convs):
        #out.add(
        #    nn.Conv2D(channels=channels, kernel_size=3,
        #              padding=1, activation='relu')
        #)
        out.add(nn.Conv2D(channels=channels, kernel_size=3, padding=1))
        out.add(nn.BatchNorm(axis=1))
        out.add(nn.Activation(activation='relu'))
    out.add(nn.MaxPool2D(pool_size=2, strides=2))
    return out

In [11]:
#将这些块堆起来
def vgg_stack(architecture):
    out = nn.Sequential()
    for (num_convs, channels) in architecture:
        out.add(vgg_block(num_convs, channels))
    return out

In [12]:
#最简单的一个VGG结构，它有8个卷积层，和跟Alexnet一样的3个全连接层。这个网络又称VGG 11
num_outputs = 10
architecture_11 = ((1,64), (1,128), (2,256), (2,512), (2,512))
architecture_16 = ((2,64), (2,128), (3, 256), (3, 512), (3, 512))
architecture_19 = ((2,64), (2,128), (4, 256), (4, 512), (4, 512))
net = nn.Sequential()
with net.name_scope():
    net.add(vgg_stack(architecture_11))
    net.add(nn.Flatten())
    net.add(nn.Dense(4096, activation="relu"))
    net.add(nn.Dropout(.5))
    net.add(nn.Dense(4096, activation="relu"))
    net.add(nn.Dropout(.5))
    net.add(nn.Dense(num_outputs))

In [13]:
#Alexnet的训练代码一样除了我们只将图片扩大到96×9696×96来节省些计算，和默认使用稍微大点的学习率
import sys
sys.path.append('..')
import utils
from mxnet import gluon
from mxnet import init

train_data, test_data = utils.load_data_fashion_mnist(
    batch_size=64, resize=96)

ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.05})
utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=5)

Epoch 0. Loss: 0.683937, Train acc 0.769956, Test acc 0.770402
Epoch 1. Loss: 0.332803, Train acc 0.878665, Test acc 0.906350
Epoch 2. Loss: 0.270128, Train acc 0.901769, Test acc 0.878682
Epoch 3. Loss: 0.227210, Train acc 0.917511, Test acc 0.922771
Epoch 4. Loss: 0.200031, Train acc 0.927739, Test acc 0.899980
